In [2]:
!pip install xgboost
!pip install imblearn
!apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler,label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score, f1_score,precision_score, recall_score, roc_auc_score, average_precision_score,roc_curve, precision_recall_curve, hamming_loss,auc
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import dask.dataframe as dd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!cp "/content/drive/MyDrive/Dataset_SRIP/GothamDataset2025.7z" "/content/dataset.7z"

!7z x /content/dataset.7z -o/content/data


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 271870960 bytes (260 MiB)

Extracting archive: /content/dataset.7z
--
Path = /content/dataset.7z
Type = 7z
Physical Size = 271870960
Headers Size = 1234
Method = LZMA2:28
Solid = +
Blocks = 1

  0%    
Would you like to replace the existing file:
  Path:     /content/data/GothamDataset2025/iotsim-air-quality-1.csv
  Size:     12198548 bytes (12 MiB)
  Modified: 2025-01-29 16:49:40
with the file from archive:
  Path:     GothamDataset2025/iotsim-air-quality-1.csv
  Size:     12198548 bytes (12 MiB)
  Modified: 2025-01-29 16:49:40
? (Y)es / (N)o / (A)lways / (S)kip all / A(u)to rename all / (Q)uit? N

  0% 1 . GothamDataset2025/iotsim-air-quality-1.csv

In [6]:
folder_path = "/content/data/GothamDataset2025/*.csv"

data = dd.read_csv(folder_path, dtype=str)


In [7]:
data.head()

,frame.time,frame.len,frame.protocols,eth.src,eth.dst,ip.dst,ip.src,ip.flags,ip.ttl,ip.proto,...,tcp.dstport,tcp.flags,tcp.window_size_value,tcp.window_size_scalefactor,tcp.checksum,tcp.options,tcp.pdu.size,udp.srcport,udp.dstport,label
0,"Jan 14, 2025 18:40:22.447710000 GMT",67,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,48322.0,53.0,Benign
1,"Jan 14, 2025 18:40:22.453402000 GMT",83,eth:ethertype:ip:udp:dns,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.2,0x02,61,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,53.0,48322.0,Benign
2,"Jan 14, 2025 18:40:22.453507000 GMT",90,eth:ethertype:ip:udp:ntp,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.3,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,46343.0,123.0,Benign
3,"Jan 14, 2025 18:40:22.458119000 GMT",90,eth:ethertype:ip:udp:ntp,0c:6e:9c:16:00:00,02:42:52:d7:fa:00,192.168.18.17,192.168.0.3,0x02,61,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,123.0,46343.0,Benign
4,"Jan 14, 2025 18:40:22.560013000 GMT",76,eth:ethertype:ip:udp:dns,02:42:52:d7:fa:00,0c:6e:9c:16:00:00,192.168.0.2,192.168.18.17,0x02,64,17,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,36848.0,53.0,Benign


In [9]:
data.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 23 entries, frame.time to label
dtypes: float64(11), int64(3), string(9)

In [8]:
data.columns

Index(['frame.time', 'frame.len', 'frame.protocols', 'eth.src', 'eth.dst',
       'ip.dst', 'ip.src', 'ip.flags', 'ip.ttl', 'ip.proto', 'ip.checksum',
       'ip.tos', 'tcp.srcport', 'tcp.dstport', 'tcp.flags',
       'tcp.window_size_value', 'tcp.window_size_scalefactor', 'tcp.checksum',
       'tcp.options', 'tcp.pdu.size', 'udp.srcport', 'udp.dstport', 'label'],
      dtype='object')

In [10]:
text=data['label'].value_counts()
text.compute()

,count
label,
Mirai UDP Flooding,8897895
Merlin ICMP Flooding,57580
File Download,7196
UDP Scan,4242
Benign,12256883
Telnet Brute Force,227649
Mirai GRE Flooding,5911401
Merlin UDP Flooding,29996
CoAP Amplification,274837


In [ ]:
def head_n(partition, n=10000):
    return partition.head(n)
subset = data.groupby("label").apply(head_n, n=10000, meta=data)

# Bring the smaller subset into memory (safe now)
subset_pd = subset.compute()

print(subset_pd['label'].value_counts())

In [ ]:
data=data.drop_duplicates()

In [ ]:
data=data.drop(columns=["pkSeqID","stime","flgs","proto","saddr","sport","daddr","dport","state","ltime","seq"])

In [ ]:
data.isnull().sum()

In [10]:
data.head()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------------+--------+----------+
| Column       | Found  | Expected |
+--------------+--------+----------+
| tcp.checksum | object | float64  |
| tcp.flags    | object | float64  |
| tcp.options  | object | float64  |
+--------------+--------+----------+

The following columns also raised exceptions on conversion:

- tcp.checksum
  ValueError("could not convert string to float: '0xc4fa'")
- tcp.flags
  ValueError("could not convert string to float: '0x0002'")
- tcp.options
  ValueError("could not convert string to float: '020405b40402080ac4c967b60000000001030307'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'tcp.checksum': 'object',
       'tcp.flags': 'object',
       'tcp.options': 'object'}

to the call to `read_csv`/`read_table`.